<a href="https://colab.research.google.com/github/snaiws/Project_5/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
'''
대충 api 써보기 위해 구글에서 긁어온 코드
'''

import requests
from urllib import parse

def encrypt(DEVELOPMENTAPIKEY,summonerName):
    encodingSummonerName = parse.quote(summonerName)
    APIURL = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + encodingSummonerName
    headers = {
        "Origin": "https://developer.riotgames.com",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": DEVELOPMENTAPIKEY,
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36"
        }
    res = requests.get(APIURL, headers=headers)
    data = res.json()
    return data["id"]
print(encrypt("RGAPI-e8ae94ad-c964-44f3-850a-1ca0e1b4acb5","HIDE ON BUSH"))

xAW4qvXEbYbqNinDjL_OEXIPkI6a4W-CGrrHKrhVOwNa3Q


In [11]:

def getTier(DEVELOPMENTAPIKEY,summonerName):
    encryptedId = encrypt(DEVELOPMENTAPIKEY,summonerName)
    headers = {
        "Origin": "https://developer.riotgames.com",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "X-Riot-Token": DEVELOPMENTAPIKEY,
        "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36"
        }
    APIURL = "https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/" + encryptedId
    res = requests.get(APIURL, headers=headers)
    data = res.json()
    return data[0]["tier"]

DEVELOPMENTAPIKEY = "RGAPI-e8ae94ad-c964-44f3-850a-1ca0e1b4acb5"
summonerName = "HIDE ON BUSH"

print(getTier(DEVELOPMENTAPIKEY,summonerName))

DIAMOND


In [12]:
DEVELOPMENTAPIKEY='RGAPI-e8ae94ad-c964-44f3-850a-1ca0e1b4acb5'
url='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/III?page=1'
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": DEVELOPMENTAPIKEY
}

res = requests.get(url, headers=headers)
data = res.json()
data

{'status': {'message': 'Bad request - Division is invalid',
  'status_code': 400}}

In [15]:
url='https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page=1&api_key=RGAPI-e8ae94ad-c964-44f3-850a-1ca0e1b4acb5'
res = requests.get(url)
data = res.json()
data

[{'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '7bcfad3d-8763-499e-9b3c-ca900cc153f3',
  'leaguePoints': 17,
  'losses': 137,
  'queueType': 'RANKED_SOLO_5x5',
  'rank': 'I',
  'summonerId': '_xRTXbnY1riJwMr4CQl1UHEA_sNWb0cbb_Pvg5IX-fSjniPW',
  'summonerName': 'Livefastdieslow',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 167},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': '2667d1a2-5064-42bf-9b4e-6afdcb8e7b37',
  'leaguePoints': 100,
  'losses': 255,
  'miniSeries': {'losses': 2, 'progress': 'LLWNN', 'target': 3, 'wins': 1},
  'queueType': 'RANKED_SOLO_5x5',
  'rank': 'I',
  'summonerId': 'egQpllR9Hhh4zWbhjw8fI3CXt4PYyRxBgjTxr9VX6QHyV-Rt',
  'summonerName': '종결미드라인',
  'tier': 'DIAMOND',
  'veteran': False,
  'wins': 297},
 {'freshBlood': False,
  'hotStreak': False,
  'inactive': False,
  'leagueId': 'a952afaf-7657-4c11-9cda-bc2f62342e86',
  'leaguePoints': 36,
  'losses': 74,
  'queueType': 'RANKED_SOLO_5x5',
